<a href="https://colab.research.google.com/github/gsdos1984-sudo/EPPsimulatorby-Miguel-verastegui/blob/main/golden_recepies_bymiguelverastegui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ======= Colab cell: Golden recipes EPP (Gradio) =======
!pip -q install gradio pandas openpyxl plotly

import os
from datetime import datetime
from typing import Tuple, Dict, Any
import pandas as pd
import plotly.express as px
import gradio as gr

EXCEL_PATH = "epp_golden_recipes.xlsx"
SHEET_DATA = "Recipes"
SHEET_PRINT = "Printable"

COLUMNS = [
    "ts","author","rev",
    "part_number","part_name","customer","bead_grade","hb_grade","batch_lot",
    "air_pre_bar","steam_pre_bar","water_pre_bar","cooling_tower_c",
    "cycle_time_s","pre_blowing_s",
    "fill1_time_s","fill1_sp_bar","fill2_time_s","fill2_sp_bar","blowback_s",
    "xsteam_fix_sv_bar","xsteam_fix_pv_bar","xsteam_mob_sv_bar","xsteam_mob_pv_bar",
    "autoclave_press_bar","autoclave_time_s",
    "fixed_xsteam_s","fixed_xsteam_bar","mov_xsteam_s","mov_xsteam_bar",
    "cool_fit_time_s","cool_moldin_time_s","stabilization_time_s",
    "purge_air_s","drain_s","foam_in_s","foam_out_s","vac_valve_s",
    "crack_air_bar","ejector_air_bar",
    "wet_min_g","wet_nom_g","wet_max_g","dry_min_g","dry_nom_g","dry_max_g",
    "comments",
]

def ensure_excel():
    if not os.path.exists(EXCEL_PATH):
        df = pd.DataFrame(columns=COLUMNS)
        with pd.ExcelWriter(EXCEL_PATH, engine="openpyxl") as wr:
            df.to_excel(wr, index=False, sheet_name=SHEET_DATA)
            pd.DataFrame().to_excel(wr, index=False, sheet_name=SHEET_PRINT)

def load_df() -> pd.DataFrame:
    ensure_excel()
    try:
        return pd.read_excel(EXCEL_PATH, sheet_name=SHEET_DATA)
    except Exception:
        return pd.DataFrame(columns=COLUMNS)

def save_df(df: pd.DataFrame):
    ensure_excel()
    with pd.ExcelWriter(EXCEL_PATH, engine="openpyxl") as wr:
        df.to_excel(wr, index=False, sheet_name=SHEET_DATA)
        pd.DataFrame().to_excel(wr, index=False, sheet_name=SHEET_PRINT)

def upsert_recipe(row: Dict[str, Any]) -> Tuple[str, pd.DataFrame]:
    df = load_df()
    mask = (df["part_number"].astype(str)==str(row["part_number"])) & (df["rev"].astype(str)==str(row["rev"]))
    row_df = pd.DataFrame([row], columns=COLUMNS)
    if mask.any():
        df.loc[mask,:] = row_df.values
        msg = "♻️ Receta actualizada"
    else:
        df = pd.concat([df, row_df], ignore_index=True)
        msg = "✅ Receta creada"
    save_df(df)
    return msg, df.sort_values(["part_number","rev"]).reset_index(drop=True)

def delete_recipe(part_number: str, rev: str) -> Tuple[str, pd.DataFrame]:
    df = load_df()
    mask = (df["part_number"].astype(str)==str(part_number)) & (df["rev"].astype(str)==str(rev))
    if not mask.any():
        return "No existe esa receta.", df
    df = df[~mask].reset_index(drop=True)
    save_df(df)
    return "🗑️ Receta eliminada", df

def get_part_numbers() -> list:
    df = load_df()
    return sorted(df["part_number"].dropna().astype(str).unique().tolist()) if not df.empty else []

def load_recipe(part_number: str, rev: str) -> Dict[str, Any]:
    df = load_df()
    mask = (df["part_number"].astype(str)==str(part_number)) & (df["rev"].astype(str)==str(rev))
    return df[mask].iloc[0].to_dict() if (not df.empty and mask.any()) else {}

def latest_rev_for_part(part_number: str) -> str:
    df = load_df()
    dfx = df[df["part_number"].astype(str)==str(part_number)]
    if dfx.empty: return ""
    if "ts" in dfx.columns:
        dfx["ts"] = pd.to_datetime(dfx["ts"], errors="coerce")
        dfx = dfx.sort_values("ts")
    return str(dfx.iloc[-1].get("rev",""))

def make_printable_sheet(row: Dict[str, Any]) -> str:
    labels = {
        "Part Number": row.get("part_number",""),
        "Part Name": row.get("part_name",""),
        "Customer": row.get("customer",""),
        "Bead Grade": row.get("bead_grade",""),
        "HB Grade": row.get("hb_grade",""),
        "Batch / Lot": row.get("batch_lot",""),
        "Rev": row.get("rev",""),
        "Autor": row.get("author",""),
        "Fecha": row.get("ts",""),
        "Air pre (bar)": row.get("air_pre_bar",""),
        "Steam pre (bar)": row.get("steam_pre_bar",""),
        "Water pre (bar)": row.get("water_pre_bar",""),
        "Cooling tower (°C)": row.get("cooling_tower_c",""),
        "Cycle time (s)": row.get("cycle_time_s",""),
        "Pre-Blowing (s)": row.get("pre_blowing_s",""),
        "Filling 1 time (s)": row.get("fill1_time_s",""),
        "Filling 1 SP (bar)": row.get("fill1_sp_bar",""),
        "Filling 2 time (s)": row.get("fill2_time_s",""),
        "Filling 2 SP (bar)": row.get("fill2_sp_bar",""),
        "Blowback (s)": row.get("blowback_s",""),
        "X-Steam FIX SV (bar)": row.get("xsteam_fix_sv_bar",""),
        "X-Steam FIX PV (bar)": row.get("xsteam_fix_pv_bar",""),
        "X-Steam MOV SV (bar)": row.get("xsteam_mob_sv_bar",""),
        "X-Steam MOV PV (bar)": row.get("xsteam_mob_pv_bar",""),
        "Autoclave press (bar)": row.get("autoclave_press_bar",""),
        "Autoclave time (s)": row.get("autoclave_time_s",""),
        "FIXED X-Steam (s)": row.get("fixed_xsteam_s",""),
        "FIXED X-Steam (bar)": row.get("fixed_xsteam_bar",""),
        "MOV X-Steam (s)": row.get("mov_xsteam_s",""),
        "MOV X-Steam (bar)": row.get("mov_xsteam_bar",""),
        "Cooling FIT (s)": row.get("cool_fit_time_s",""),
        "Cooling MOLD-IN (s)": row.get("cool_moldin_time_s",""),
        "Stabilization (s)": row.get("stabilization_time_s",""),
        "Purge Air (s)": row.get("purge_air_s",""),
        "Drain (s)": row.get("drain_s",""),
        "Foam-in (s)": row.get("foam_in_s",""),
        "Foam-out (s)": row.get("foam_out_s",""),
        "Vac Valve (s)": row.get("vac_valve_s",""),
        "Crack Air (bar)": row.get("crack_air_bar",""),
        "Ejector Air (bar)": row.get("ejector_air_bar",""),
        "Wet Min (g)": row.get("wet_min_g",""),
        "Wet Nom (g)": row.get("wet_nom_g",""),
        "Wet Max (g)": row.get("wet_max_g",""),
        "Dry Min (g)": row.get("dry_min_g",""),
        "Dry Nom (g)": row.get("dry_nom_g",""),
        "Dry Max (g)": row.get("dry_max_g",""),
        "Comments": row.get("comments",""),
    }
    dfp = pd.DataFrame(list(labels.items()), columns=["Field","Value"])
    df = load_df()
    with pd.ExcelWriter(EXCEL_PATH, engine="openpyxl", mode="a", if_sheet_exists="replace") as wr:
        df.to_excel(wr, index=False, sheet_name=SHEET_DATA)
        dfp.to_excel(wr, index=False, sheet_name=SHEET_PRINT)
    return f"Hoja '{SHEET_PRINT}' generada en {EXCEL_PATH}"

def calc_stabilization(cool_fit_time_s, cool_moldin_time_s) -> float:
    try:
        a = float(cool_fit_time_s or 0); b = float(cool_moldin_time_s or 0)
        return round(0.5 * max(a, b), 1)
    except:
        return 0.0

def on_part_change(part_number):
    df = load_df()
    dfx = df[df["part_number"].astype(str)==str(part_number)]
    if dfx.empty: return "", []
    revs = dfx["rev"].astype(str).unique().tolist()
    last = latest_rev_for_part(part_number)
    return last, sorted(revs)

def load_for_edit(part_number, rev):
    d = load_recipe(part_number, rev)
    if not d:
        # 1 mensaje + 47 valores vacíos para alinear con outputs
        return ("No encontrado.",) + tuple([""]*47)
    return (
        "Receta cargada.",
        d.get("part_number",""), d.get("part_name",""), d.get("customer",""), d.get("bead_grade",""),
        d.get("hb_grade",""), d.get("batch_lot",""),
        d.get("air_pre_bar",None), d.get("steam_pre_bar",None), d.get("water_pre_bar",None), d.get("cooling_tower_c",None),
        d.get("cycle_time_s",None), d.get("pre_blowing_s",None),
        d.get("fill1_time_s",None), d.get("fill1_sp_bar",None), d.get("fill2_time_s",None), d.get("fill2_sp_bar",None), d.get("blowback_s",None),
        d.get("xsteam_fix_sv_bar",None), d.get("xsteam_fix_pv_bar",None), d.get("xsteam_mob_sv_bar",None), d.get("xsteam_mob_pv_bar",None),
        d.get("autoclave_press_bar",None), d.get("autoclave_time_s",None),
        d.get("fixed_xsteam_s",None), d.get("fixed_xsteam_bar",None), d.get("mov_xsteam_s",None), d.get("mov_xsteam_bar",None),
        d.get("cool_fit_time_s",None), d.get("cool_moldin_time_s",None), d.get("stabilization_time_s",None),
        d.get("purge_air_s",None), d.get("drain_s",None), d.get("foam_in_s",None), d.get("foam_out_s",None), d.get("vac_valve_s",None),
        d.get("crack_air_bar",None), d.get("ejector_air_bar",None),
        d.get("wet_min_g",None), d.get("wet_nom_g",None), d.get("wet_max_g",None),
        d.get("dry_min_g",None), d.get("dry_nom_g",None), d.get("dry_max_g",None),
        d.get("comments",""), d.get("rev",""), d.get("author",""), d.get("ts","")
    )

def submit_form(
    author, rev,
    part_number, part_name, customer, bead_grade, hb_grade, batch_lot,
    air_pre_bar, steam_pre_bar, water_pre_bar, cooling_tower_c,
    cycle_time_s, pre_blowing_s,
    fill1_time_s, fill1_sp_bar, fill2_time_s, fill2_sp_bar, blowback_s,
    xsteam_fix_sv_bar, xsteam_fix_pv_bar, xsteam_mob_sv_bar, xsteam_mob_pv_bar,
    autoclave_press_bar, autoclave_time_s,
    fixed_xsteam_s, fixed_xsteam_bar, mov_xsteam_s, mov_xsteam_bar,
    cool_fit_time_s, cool_moldin_time_s, stabilization_time_s,
    purge_air_s, drain_s, foam_in_s, foam_out_s, vac_valve_s,
    crack_air_bar, ejector_air_bar,
    wet_min_g, wet_nom_g, wet_max_g, dry_min_g, dry_nom_g, dry_max_g,
    comments, ts
):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    stab = stabilization_time_s if stabilization_time_s not in (None,"") else calc_stabilization(cool_fit_time_s, cool_moldin_time_s)
    row = {
        "ts": ts or now, "author": author, "rev": rev,
        "part_number": part_number, "part_name": part_name, "customer": customer, "bead_grade": bead_grade,
        "hb_grade": hb_grade, "batch_lot": batch_lot,
        "air_pre_bar": air_pre_bar, "steam_pre_bar": steam_pre_bar, "water_pre_bar": water_pre_bar, "cooling_tower_c": cooling_tower_c,
        "cycle_time_s": cycle_time_s, "pre_blowing_s": pre_blowing_s,
        "fill1_time_s": fill1_time_s, "fill1_sp_bar": fill1_sp_bar, "fill2_time_s": fill2_time_s, "fill2_sp_bar": fill2_sp_bar, "blowback_s": blowback_s,
        "xsteam_fix_sv_bar": xsteam_fix_sv_bar, "xsteam_fix_pv_bar": xsteam_fix_pv_bar, "xsteam_mob_sv_bar": xsteam_mob_sv_bar, "xsteam_mob_pv_bar": xsteam_mob_pv_bar,
        "autoclave_press_bar": autoclave_press_bar, "autoclave_time_s": autoclave_time_s,
        "fixed_xsteam_s": fixed_xsteam_s, "fixed_xsteam_bar": fixed_xsteam_bar, "mov_xsteam_s": mov_xsteam_s, "mov_xsteam_bar": mov_xsteam_bar,
        "cool_fit_time_s": cool_fit_time_s, "cool_moldin_time_s": cool_moldin_time_s, "stabilization_time_s": stab,
        "purge_air_s": purge_air_s, "drain_s": drain_s, "foam_in_s": foam_in_s, "foam_out_s": foam_out_s, "vac_valve_s": vac_valve_s,
        "crack_air_bar": crack_air_bar, "ejector_air_bar": ejector_air_bar,
        "wet_min_g": wet_min_g, "wet_nom_g": wet_nom_g, "wet_max_g": wet_max_g,
        "dry_min_g": dry_min_g, dry_nom_g: dry_nom_g, "dry_max_g": dry_max_g,
        "comments": comments,
    }
    msg, df = upsert_recipe(row)
    by_bead = df.groupby("bead_grade")["part_number"].count().reset_index().rename(columns={"part_number":"recipes"}) if not df.empty else pd.DataFrame(columns=["bead_grade","recipes"])
    by_cust = df.groupby("customer")["part_number"].count().reset_index().rename(columns={"part_number":"recipes"}) if not df.empty else pd.DataFrame(columns=["customer","recipes"])
    fig1 = px.bar(by_bead, x="bead_grade", y="recipes", title="Recetas por Bead") if not by_bead.empty else None
    fig2 = px.bar(by_cust, x="customer", y="recipes", title="Recetas por Cliente") if not by_cust.empty else None
    return msg, df.sort_values(["part_number","rev"]), fig1, fig2

def delete_current(part_number, rev):
    msg, df = delete_recipe(part_number, rev)
    fig1 = px.bar(df.groupby("bead_grade")["part_number"].count().reset_index().rename(columns={"part_number":"recipes"}),
                  x="bead_grade", y="recipes", title="Recetas por Bead") if not df.empty else None
    fig2 = px.bar(df.groupby("customer")["part_number"].count().reset_index().rename(columns={"part_number":"recipes"}),
                  x="customer", y="recipes", title="Recetas por Cliente") if not df.empty else None
    return msg, df, fig1, fig2

def export_printable(part_number, rev):
    d = load_recipe(part_number, rev)
    if not d: return "No se encontró la receta."
    return make_printable_sheet(d)

ensure_excel()
pn_choices = get_part_numbers()

with gr.Blocks(theme=gr.themes.Soft(), title="Golden recipes EPP by Miguel Verastegui") as demo:
    gr.Markdown("## 🟡 Golden recipes EPP by Miguel Verastegui")
    gr.Markdown("Captura, consulta y controla recetas doradas en Excel.")

    with gr.Tab("✍️ Captura / Edición"):
        with gr.Row():
            part_number = gr.Textbox(label="Part Number", placeholder="Ej. MXS-6G-FFL", scale=2)
            rev = gr.Textbox(label="Revision", placeholder="Ej. A, B, C, 1, 2...", scale=1)
            author = gr.Textbox(label="Autor", value="Miguel Verastegui", scale=1)
        with gr.Row():
            part_name = gr.Textbox(label="Part Name / Description", scale=2)
            customer  = gr.Textbox(label="Customer", scale=1)
            bead_grade= gr.Dropdown(choices=["15","22","35","42"], value="35", label="Bead grade (g/L nominal)", scale=1)
        with gr.Row():
            hb_grade  = gr.Textbox(label="HB Grade", scale=1)
            batch_lot = gr.Textbox(label="Batch / Lot", scale=1)
            ts        = gr.Textbox(label="Timestamp (auto)", value="", interactive=False, scale=1)

        gr.Markdown("### ⚙️ Utilidades")
        with gr.Row():
            air_pre_bar    = gr.Number(label="Air pre (bar)", value=5.0)
            steam_pre_bar  = gr.Number(label="Steam pre (bar)", value=7.0)
            water_pre_bar  = gr.Number(label="Water pre (bar)", value=3.0)
            cooling_tower_c= gr.Number(label="Cooling tower (°C)", value=25)
        with gr.Row():
            cycle_time_s   = gr.Number(label="Cycle time (s)", value=150)
            pre_blowing_s  = gr.Number(label="Pre-Blowing (s)", value=0)

        gr.Markdown("### 🧵 Filling")
        with gr.Row():
            fill1_time_s = gr.Number(label="Filling 1 time (s)", value=3.5)
            fill1_sp_bar = gr.Number(label="Filling 1 SP (bar)", value=3.5)
            fill2_time_s = gr.Number(label="Filling 2 time (s)", value=2.8)
            fill2_sp_bar = gr.Number(label="Filling 2 SP (bar)", value=2.8)
        blowback_s = gr.Number(label="Blowback (s)", value=0)

        gr.Markdown("### 🔥 X-Steam / Reguladores")
        with gr.Row():
            xsteam_fix_sv_bar = gr.Number(label="X-Steam FIX SV (bar)", value=5.5)
            xsteam_fix_pv_bar = gr.Number(label="X-Steam FIX PV (bar)", value=5.7)
            xsteam_mob_sv_bar = gr.Number(label="X-Steam MOV SV (bar)", value=5.5)
            xsteam_mob_pv_bar = gr.Number(label="X-Steam MOV PV (bar)", value=5.6)

        gr.Markdown("### 🛢️ Autoclave / Aging")
        with gr.Row():
            autoclave_press_bar = gr.Number(label="Autoclave press (bar)", value=5.5)
            autoclave_time_s    = gr.Number(label="Autoclave time (s)", value=300)

        gr.Markdown("### 🔁 FIXED / MOV – Tiempos de vapor")
        with gr.Row():
            fixed_xsteam_s  = gr.Number(label="FIXED X-Steam (s)", value=5)
            fixed_xsteam_bar= gr.Number(label="FIXED X-Steam (bar)", value=5.5)
            mov_xsteam_s    = gr.Number(label="MOV X-Steam (s)", value=5)
            mov_xsteam_bar  = gr.Number(label="MOV X-Steam (bar)", value=5.5)

        gr.Markdown("### ❄️ Cooling / Estabilización")
        with gr.Row():
            cool_fit_time_s    = gr.Number(label="Cooling FIT time (s)", value=60)
            cool_moldin_time_s = gr.Number(label="Cooling MOLD-IN time (s)", value=90)
            stabilization_time_s = gr.Number(label="Stabilization time (s) (50% del mayor)", value=45)

        gr.Markdown("### 🌬️ Válvulas / Secuencias")
        with gr.Row():
            purge_air_s = gr.Number(label="Purge Air (s)", value=3)
            drain_s     = gr.Number(label="Drain (s)", value=3)
            foam_in_s   = gr.Number(label="Foam-in (s)", value=3)
            foam_out_s  = gr.Number(label="Foam-out (s)", value=3)
            vac_valve_s = gr.Number(label="Vac Valve (s)", value=3)
        with gr.Row():
            crack_air_bar  = gr.Number(label="Crack Air (bar)", value=0.0)
            ejector_air_bar= gr.Number(label="Ejector Air (bar)", value=0.0)

        gr.Markdown("### ⚖️ Weights (targets)")
        with gr.Row():
            wet_min_g = gr.Number(label="Wet Min (g)", value=None)
            wet_nom_g = gr.Number(label="Wet Nom (g)", value=None)
            wet_max_g = gr.Number(label="Wet Max (g)", value=None)
            dry_min_g = gr.Number(label="Dry Min (g)", value=None)
            dry_nom_g = gr.Number(label="Dry Nom (g)", value=None)
            dry_max_g = gr.Number(label="Dry Max (g)", value=None)

        comments = gr.Textbox(label="Comentarios", lines=3)

        status = gr.Textbox(label="Estado", interactive=False)
        with gr.Row():
            btn_save   = gr.Button("💾 Guardar / Actualizar", variant="primary")
            btn_delete = gr.Button("🗑️ Eliminar receta")
            btn_print  = gr.Button("🖨️ Generar hoja Printable (Excel)")
            btn_calcst = gr.Button("🧮 Calcular Stabilization = 50% del mayor")

    with gr.Tab("🔎 Buscar / Cargar"):
        with gr.Row():
            sel_part = gr.Dropdown(choices=pn_choices, label="Part Number (existentes)", allow_custom_value=True)
            sel_rev  = gr.Dropdown(choices=[], label="Revisión (existentes)", allow_custom_value=True)
            btn_load = gr.Button("Cargar para editar")
            load_msg = gr.Textbox(label="Estado de carga", interactive=False)

    with gr.Tab("📊 Tablero / Export"):
        table = gr.Dataframe(label="Recetas", wrap=True, row_count=(5, "dynamic"))
        fig_bead = gr.Plot()
        fig_cust = gr.Plot()
        btn_refresh = gr.Button("🔄 Refrescar tabla")
        btn_download = gr.File(label="Descargar Excel (abre y guarda)", interactive=False)

    def calc_and_update(a, b, current):
        return calc_stabilization(a, b)
    btn_calcst.click(calc_and_update, inputs=[cool_fit_time_s, cool_moldin_time_s, stabilization_time_s],
                     outputs=[stabilization_time_s])

    btn_save.click(
        submit_form,
        inputs=[
            author, rev,
            part_number, part_name, customer, bead_grade, hb_grade, batch_lot,
            air_pre_bar, steam_pre_bar, water_pre_bar, cooling_tower_c,
            cycle_time_s, pre_blowing_s,
            fill1_time_s, fill1_sp_bar, fill2_time_s, fill2_sp_bar, blowback_s,
            xsteam_fix_sv_bar, xsteam_fix_pv_bar, xsteam_mob_sv_bar, xsteam_mob_pv_bar,
            autoclave_press_bar, autoclave_time_s,
            fixed_xsteam_s, fixed_xsteam_bar, mov_xsteam_s, mov_xsteam_bar,
            cool_fit_time_s, cool_moldin_time_s, stabilization_time_s,
            purge_air_s, drain_s, foam_in_s, foam_out_s, vac_valve_s,
            crack_air_bar, ejector_air_bar,
            wet_min_g, wet_nom_g, wet_max_g, dry_min_g, dry_nom_g, dry_max_g,
            comments, ts
        ],
        outputs=[status, table, fig_bead, fig_cust]
    )

    btn_delete.click(delete_current, inputs=[part_number, rev], outputs=[status, table, fig_bead, fig_cust])

    btn_print.click(export_printable, inputs=[part_number, rev], outputs=[status])

    def fill_revs(pn):
        last, revs = on_part_change(pn)
        return gr.update(choices=revs, value=last)
    sel_part.change(fill_revs, inputs=[sel_part], outputs=[sel_rev])

    # 👉 CORRECCIÓN: outputs como LISTA (no set) y mismo orden de valores que devuelve load_for_edit
    btn_load.click(
        load_for_edit,
        inputs=[sel_part, sel_rev],
        outputs=[
            load_msg,
            part_number, part_name, customer, bead_grade, hb_grade, batch_lot,
            air_pre_bar, steam_pre_bar, water_pre_bar, cooling_tower_c,
            cycle_time_s, pre_blowing_s,
            fill1_time_s, fill1_sp_bar, fill2_time_s, fill2_sp_bar, blowback_s,
            xsteam_fix_sv_bar, xsteam_fix_pv_bar, xsteam_mob_sv_bar, xsteam_mob_pv_bar,
            autoclave_press_bar, autoclave_time_s,
            fixed_xsteam_s, fixed_xsteam_bar, mov_xsteam_s, mov_xsteam_bar,
            cool_fit_time_s, cool_moldin_time_s, stabilization_time_s,
            purge_air_s, drain_s, foam_in_s, foam_out_s, vac_valve_s,
            crack_air_bar, ejector_air_bar,
            wet_min_g, wet_nom_g, wet_max_g, dry_min_g, dry_nom_g, dry_max_g,
            comments, rev, author, ts
        ]
    )

    def refresh_table():
        return load_df().sort_values(["part_number","rev"]).reset_index(drop=True)
    btn_refresh.click(refresh_table, outputs=[table])

    def expose_excel():
        ensure_excel()
        return EXCEL_PATH
    # remove the click event binding for btn_download
    # btn_download.click(expose_excel, outputs=[btn_download])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec92a9f828f6ac2532.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
